In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astropy.convolution import Gaussian2DKernel,convolve
from astropy.io import fits
from scipy.ndimage import zoom

In [2]:
def count2flux(img_count,replacer=1e-10,mag_const=24.37,
               lamda=6563,delta_lamda=250,z=2.3):
    '''
    convert counts map to flux map.
    lamda and delta_lamda in unit of \AA
    '''
    #replace negtive value with positive
    img_count[img_count<=0]=replacer
    
    mag=-2.5*np.log10(img_count)+mag_const
    flux_nu=10**(-(mag+48.6)/2.5)
    flux_lamda=flux_nu*const.c.to(u.AA/u.second).value/((lamda*(1+z))**2)
    flux=flux_lamda*delta_lamda*u.erg/(u.second*(u.cm**2))
    
    return flux

def pix2world(shape,delta,origin_pix,origin_world):
    '''
    convert pixel coordinate to
    ra,dec, origin_pix and origin_world
    is the coordinate of origin in 
    different frame. They are both 1-d
    array with two elements
    return two 1-d arrays of ra and 
    dec respectively
    '''
    
    x,y=np.arange(shape[1]),np.arange(shape[0])
    X,Y=np.meshgrid(x,y)
    ra_map=(origin_pix[0]-X)*delta[0]+origin_world[0]
    dec_map=(origin_pix[1]-Y)*delta[1]+origin_world[1]
    
    return ra_map.mean(axis=0), dec_map.mean(axis=1)

def img_fix(img,dec,ra,slopes,offsets):
    '''
    set the value of region ignored to be zero
    '''
    img_fixed=img.copy()
    ra_grid,dec_grid=np.meshgrid(ra,dec)
    mask1=~(slopes[0]*ra_grid+offsets[0]>=dec_grid)
    mask2=~(slopes[0]*ra_grid+offsets[1]<=dec_grid)
    
    mask3=~(slopes[1]*ra_grid+offsets[2]>=dec_grid)
    mask4=~(slopes[1]*ra_grid+offsets[3]<=dec_grid)
    img_fixed[mask1|mask2|mask3|mask4]=0
    
    return img_fixed

In [3]:
dic='/Users/shiwuzhang/WS/ASTRO/MAMMOTH_KCWI/results/'
h_fits=fits.open(dic+'MORICS_geotran_cut.fits')

In [4]:
H_flux=count2flux(h_fits[0].data).value

In [5]:
delta=[7.107805249e-05,7.199198278e-05]
origin_pix=[145,141]
origin_world=[220.352093,40.0526795]
ra,dec=pix2world(H_flux.shape,delta,origin_pix,origin_world)
delta_ra=(ra-origin_world[0])*u.deg.to(u.arcsecond)
delta_dec=(dec-origin_world[1])*u.deg.to(u.arcsecond)

### sum subru image to fit kcwi image

In [6]:
img_fixed=img_fix(H_flux,delta_dec,delta_ra,[0.28,-1/0.28],[30,-30,100,-100])
img_fixed=img_fixed[70:-70,70:-70]
delta_ra=delta_ra[70:-70]
delta_dec=delta_dec[70:-70]

In [7]:
H_f=img_fixed.copy()[60:110,50:100].sum()*u.erg/(u.s*(u.cm**2))
D=18528.4*u.Mpc
L_h=(H_f*4*np.pi*D**2).to(u.erg/u.s)
L_h

<Quantity 7.10652245e+43 erg / s>

In [8]:
img_new=np.add.reduceat(img_fixed,range(0,len(delta_ra),6),axis=0)
delta_dec=np.add.reduceat(delta_dec,range(0,len(delta_dec),6))/6
delta_dec[-1]=delta_dec[-2]+-1.55502683
delta_dec-=2.7
delta_ra-=0.5

### interpolate and smooth

In [9]:
img_inter=zoom(img_new,[20,3])
delta_dec_inter=zoom(delta_dec,20)
delta_ra_inter=zoom(delta_ra,3)

In [10]:
kernel=Gaussian2DKernel(x_stddev=3,y_stddev=3,x_size=9,y_size=9)
img_smooth=convolve(img_inter,kernel)

### background estimation

In [11]:
print('background = '+str(img_smooth[:200,:200].std()))

background = 1.1877882e-18


### extract the emission

In [12]:
noise=img_smooth[:200,:200].std()
snr_img=img_smooth/noise
img_smooth[snr_img<3]=0

#cut image
mask=np.zeros_like(snr_img)
mask=mask.astype(np.bool)
mask[150:360,171:282]=np.True_
img_smooth[~mask]=0
img_smooth[150:200,150:220]=0
img_smooth[320:400,240:300]=0
img_smooth[150:220,250:300]=0

In [14]:
savdic='/Users/shiwuzhang/WS/ASTRO/MAMMOTH_KCWI/'
np.savetxt(savdic+'H-alpha_intensity.txt',img_smooth)
np.savetxt(savdic+'H-alpha_ra.txt',delta_ra_inter)
np.savetxt(savdic+'H-alpha_dec.txt',delta_dec_inter)
np.savetxt(savdic+'H-alpha_snr.txt',snr_img)